In [ ]:
!nvidia-smi

In [ ]:
%pip install wandb

In [ ]:
import wandb
wandb.login()

In [ ]:
%env WANDB_PROJECT=combined_bengali_lora_odiagen-v0

In [ ]:
!pip install -q bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q git+https://github.com/huggingface/peft.git

In [ ]:
import os
import sys

import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers

assert (
    "LlamaTokenizer" in transformers._import_structure["models.llama"]
), "LLaMA is now in HuggingFace's main branch.\nPlease reinstall it: pip uninstall transformers && pip install git+https://github.com/huggingface/transformers.git"
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import (
    prepare_model_for_int8_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict
)

In [ ]:
# optimized for RTX 4090. for larger GPUs, increase some of these?
MICRO_BATCH_SIZE = 32  # this could actually be 5 but i like powers of 2
BATCH_SIZE = 64
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 6  # we don't always need 3 tbh
LEARNING_RATE = 3e-4  # the Karpathy constant
CUTOFF_LEN = 256  # 256 accounts for about 96% of the data
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
VAL_SET_SIZE = 2000
TARGET_MODULES = [
    "q_proj",
    "k_proj",
    "v_proj",
    "down_proj",
    "gate_proj",
    "up_proj",
]
train_on_inputs = True
add_eos_token = False

# DATA_PATH = "/content/odia_alpaca_qa_data.json"
OUTPUT_DIR = "/content/drive/MyDrive/odia_checkpoint/A100_combined_lora_odia_v1"

In [ ]:
device_map = "auto"
world_size = int(os.environ.get("WORLD_SIZE", 1))
ddp = world_size != 1
if ddp:
    device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
    GRADIENT_ACCUMULATION_STEPS = GRADIENT_ACCUMULATION_STEPS // world_size

In [ ]:
# import gc
# import torch
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
model = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map=device_map,
    torch_dtype=torch.float16
)
tokenizer = LlamaTokenizer.from_pretrained(
    "decapoda-research/llama-7b-hf"
)

tokenizer.pad_token_id = (
        0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"  # Allow batched inference

model = prepare_model_for_int8_training(model)

In [ ]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
# tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
data = load_dataset("OdiaGenAI/all_combined_bengali_252k")

In [ ]:
model.print_trainable_parameters()  # Be more transparent about the % of trainable params.

In [ ]:
def generate_prompt(instruction, input, label):

  if input:
    res = f"""### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"""
  else:
    res = f"""### Instruction:\n{instruction}\n\n### Response:\n"""

  if label:
    res = f"{res}{label}"

  return res



def tokenize(prompt, add_eos_token=True):
  # there's probably a way to do this with the tokenizer settings
  # but again, gotta move fast
  result = tokenizer(
      prompt,
      truncation=True,
      max_length=CUTOFF_LEN,
      padding=False,
      return_tensors=None,
  )
  if (
      result["input_ids"][-1] != tokenizer.eos_token_id
      and len(result["input_ids"]) < CUTOFF_LEN
      and add_eos_token
  ):
      result["input_ids"].append(tokenizer.eos_token_id)
      result["attention_mask"].append(1)

  result["labels"] = result["input_ids"].copy()

  return result

In [ ]:
def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(
      data_point["instruction"],
      data_point["input"],
      data_point["output"],
  )
  tokenized_full_prompt = tokenize(full_prompt)
  if not train_on_inputs:
      user_prompt = generate_prompt(
          data_point["instruction"], data_point["input"]
      )
      tokenized_user_prompt = tokenize(
          user_prompt, add_eos_token=add_eos_token
      )
      user_prompt_len = len(tokenized_user_prompt["input_ids"])

      if add_eos_token:
          user_prompt_len -= 1

      tokenized_full_prompt["labels"] = [
          -100
      ] * user_prompt_len + tokenized_full_prompt["labels"][
          user_prompt_len:
      ]  # could be sped up, probably
  return tokenized_full_prompt

In [ ]:
dataset = data.remove_columns("data_source")

In [ ]:
dataset['train']

In [ ]:
if VAL_SET_SIZE > 0:
    train_val = dataset["train"].train_test_split(
        test_size=VAL_SET_SIZE, shuffle=True, seed=42
    )
    train_data = train_val["train"].shuffle().map(generate_and_tokenize_prompt)
    val_data = train_val["test"].shuffle().map(generate_and_tokenize_prompt)
else:
    train_data = data["train"].shuffle().map(generate_and_tokenize_prompt)
    val_data = None

In [ ]:
train_data

In [ ]:
val_data

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=10,
        optim="adamw_torch",
        evaluation_strategy="steps" if VAL_SET_SIZE > 0 else "no",
        save_strategy="steps",
        eval_steps=100 if VAL_SET_SIZE > 0 else None,
        save_steps=100,
        output_dir=OUTPUT_DIR,
        save_total_limit=3,
        load_best_model_at_end=True if VAL_SET_SIZE > 0 else False,
        ddp_find_unused_parameters=False if ddp else None,
        group_by_length=False,
        report_to="wandb",  # enable logging to W&B
        run_name="combine_lora_bengali_v0"
    ),
    data_collator=transformers.DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        ),
)
model.config.use_cache = False

old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())
).__get__(model, type(model))

if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

In [ ]:
with torch.autocast("cuda"):
  trainer.train()

In [ ]:
model.save_pretrained("/content/")